In [65]:
# Imports
from scripts.processing import get_embeddings
from scripts.comparison import find_similar_names
from scripts.scraping import extract_texts
import json
import numpy as np
import pandas as pd
import networkx as nx
import plotly.graph_objects as go

dir = 's_names'

In [66]:
# Make articles json
articles = extract_texts(f'{dir}/names.txt')

# Write articles to JSON file
with open(f'{dir}/articles.json', 'w') as f:
    json.dump(articles, f)

Processed Sophia
Processed Sofia


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Processed Scarlett
Error processing https://www.zhihu.com/people/stella-59-15-21: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.zhihu.com/people/stella-59-15-21 on URL https://www.zhihu.com/people/stella-59-15-21
Processed Stella
Processed Sadie
Processed Sienna
Processed Summer
Processed Skylar
Processed Sloane
Processed Serena
Processed Selena
Error processing https://www.familyeducation.com/baby-names/name-meaning/seraphina: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.familyeducation.com/baby-names/name-meaning/seraphina on URL https://www.familyeducation.com/baby-names/name-meaning/seraphina
Processed Seraphina
Processed Solana
Processed Soleil
Processed Sasha
Processed Serenity


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Processed Saffron
Processed Sidra
Processed Silvana
Processed Story
Processed Sonnet
Error processing https://www.familyeducation.com/pregnancy/baby-names/girl-names-that-mean-storm: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.familyeducation.com/pregnancy/baby-names/girl-names-that-mean-storm on URL https://www.familyeducation.com/pregnancy/baby-names/girl-names-that-mean-storm
Processed Storm
Processed Shayla
Processed Shyla


...

Processed Shraddha
Processed Suzanne
Processed Suzette
Processed Suzie
Processed Suzy


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Processed Sunita
Processed Sunniva
Processed Susan
Processed Susanna
Processed Susanne
Processed Susie


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Processed Suvi
Processed Suman


In [67]:
# Read articles json and make embeddings
with open(f'{dir}/articles.json', 'r') as f:
    articles = json.load(f)

embeddings = get_embeddings(articles)

Processed Sophia with 5 texts
Processed Sofia with 4 texts
Processed Scarlett with 2 texts
Processed Stella with 1 texts
Processed Sadie with 5 texts
Processed Sienna with 5 texts
Processed Summer with 4 texts
Processed Skylar with 2 texts
Processed Sloane with 5 texts
Processed Serena with 5 texts
Processed Selena with 5 texts
Processed Seraphina with 3 texts
Processed Solana with 5 texts
Processed Soleil with 5 texts
Processed Sasha with 4 texts
Processed Serenity with 5 texts
Processed Saffron with 4 texts
Processed Sidra with 5 texts
Processed Silvana with 4 texts
Processed Story with 5 texts
Processed Sonnet with 5 texts
Processed Storm with 3 texts
Processed Shayla with 5 texts
Processed Shyla with 3 texts
Processed Sansa with 3 texts
Processed Sedona with 4 texts
Processed Sunday with 3 texts
Processed Santana with 3 texts
Processed Saraya with 3 texts
Processed Shasta with 5 texts
Processed Summerlyn with 5 texts
Processed Saint with 5 texts
Processed Sol with 5 texts
Processed

In [68]:
# Convert embeddings to dict of lists for semantic and sentiment
for key in embeddings.keys():
    embeddings[key] = {
        'semantic': np.array(embeddings[key]['semantic']).tolist(),
        'sentiment': np.array(embeddings[key]['sentiment']).tolist()
    }

# Write to json file
with open(f'{dir}/embeddings.json', 'w') as f:
    json.dump(embeddings, f, indent=4)

In [69]:
# Read embeddings from JSON file
with open(f'{dir}/embeddings.json', 'r') as f:
    embeddings = json.load(f)

# Find similar names
target_name = "Sully"
similar_names = find_similar_names(target_name, embeddings, top_k=11 , w=0.5)

print(f"Similar names to {target_name}:")
for name, score in similar_names:
    print(f"{name}: {score:.4f}")

ValueError: Sully not found in embeddings.

In [ ]:
# Calculate similarity matrix
similarity_matrix = pd.DataFrame(index=embeddings.keys(), columns=embeddings.keys())
for name1 in embeddings.keys():
    for name2 in embeddings.keys():
        if name1 != name2:
            score = np.abs(np.dot(embeddings[name1]['semantic'], embeddings[name2]['semantic']))
        else:
            score = 0

        similarity_matrix.loc[name1, name2] = score

display(similarity_matrix)



,Arya,Sully,Victoria,Erin,Hailey,Mike,Blanton,Hanjing,Francis,Benjamin,Aihan
Arya,0,0.271086,0.304895,0.050393,0.144093,0.023669,0.315711,0.332736,0.184049,0.345832,0.51089
Sully,0.271086,0,0.284549,0.052015,0.000515,0.017736,0.413166,0.313282,0.380739,0.345902,0.32932
Victoria,0.304895,0.284549,0,0.036688,0.106791,0.025678,0.283995,0.334846,0.39563,0.309696,0.261304
Erin,0.050393,0.052015,0.036688,0,0.125935,0.143076,0.066186,0.053916,0.020997,0.026255,0.046527
Hailey,0.144093,0.000515,0.106791,0.125935,0,0.107812,0.064383,0.077629,0.007813,0.092793,0.025379
Mike,0.023669,0.017736,0.025678,0.143076,0.107812,0,0.032791,0.076492,0.05371,0.13782,0.014916
Blanton,0.315711,0.413166,0.283995,0.066186,0.064383,0.032791,0,0.249334,0.330817,0.437996,0.312084
Hanjing,0.332736,0.313282,0.334846,0.053916,0.077629,0.076492,0.249334,0,0.261035,0.319649,0.409574
Francis,0.184049,0.380739,0.39563,0.020997,0.007813,0.05371,0.330817,0.261035,0,0.424551,0.210739
Benjamin,0.345832,0.345902,0.309696,0.026255,0.092793,0.13782,0.437996,0.319649,0.424551,0,0.509333


In [ ]:
names = similarity_matrix.index.tolist()
similarity_matrix = similarity_matrix.values

fig = go.Figure(data=go.Heatmap(
    z=similarity_matrix,
    x=names,
    y=names,
    colorscale='Viridis',
    zmin=0,
    zmax=1,
    colorbar=dict(title="Similarity"),
    hovertemplate="Name 1: %{y}<br>Name 2: %{x}<br>Similarity: %{z:.2f}<extra></extra>"
))

fig.update_layout(
    title="Name Similarity Heatmap",
    xaxis=dict(side="top"),
    yaxis=dict(autorange="reversed")  # so matrix reads top-left to bottom-right
)

fig.show()

In [ ]:
# Make name vector, semantic and sentimnet arrays
names = np.expand_dims(np.array(list(embeddings.keys())), 1)

sentiments = np.array([embeddings[key]['sentiment'] for key in embeddings.keys()])
semantics = np.array([embeddings[key]['semantic'] for key in embeddings.keys()])

# Save the embeddings to a csv
display(names.shape)

# Concatenate the arrays and create a DataFrame
embeddings_df = np.concatenate((sentiments, semantics), axis=1)
embeddings_df = pd.DataFrame(embeddings_df, index=names.flatten())

# Write the DataFrame to a CSV file
embeddings_df.to_csv(f'{dir}/embeddings .csv', header=False)

(11, 1)

In [ ]:
# Use KNN to cluster names
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=6)
knn.fit(embeddings_df)

# Find the nearest neighbors for the target name
distances, indices = knn.kneighbors(embeddings_df.loc[target_name].values.reshape(1, -1))

# Print the nearest neighbors not including the target name
print(f"\nNearest neighbors for {target_name}:")
for i, index in enumerate(indices[0]):
    neighbor_name = embeddings_df.index[index]
    distance = distances[0][i]
    if neighbor_name == target_name:
        continue  # Skip the target name itself
    print(f"{i}. {neighbor_name} (distance: {distance:.4f})")

    


Nearest neighbors for Sully:
1. Blanton (distance: 1.0849)
2. Aihan (distance: 1.1582)
3. Hanjing (distance: 1.1724)
4. Victoria (distance: 1.2076)
5. Francis (distance: 1.3120)
